# Specify Path of MINE_net to load

In [26]:
import os
from os import path as p

In [39]:
proj_path = p.dirname(p.abspath('MINE-Entropy'))
print(proj_path)
exp_path = p.join(proj_path, "minee")
exp_path = p.join(exp_path, "experiment")
print(exp_path)

C:\Users\chung\Dropbox\GitHub\MINE-Entropy
C:\Users\chung\Dropbox\GitHub\MINE-Entropy\minee\experiment


In [45]:
batch_size=128
population_size=2048
sub_path = p.join(exp_path, "pop={}_batch={}".format(population_size, batch_size))
print(sub_path)

C:\Users\chung\Dropbox\GitHub\MINE-Entropy\minee\experiment\pop=2048_batch=128


In [54]:
data_name="Mixed Gaussian"
vary_name="rho1"
vary_val=0.9
data_path = p.join(sub_path, "{}_{}={}".format(data_name, vary_name, vary_val))
print(data_path)

C:\Users\chung\Dropbox\GitHub\MINE-Entropy\minee\experiment\pop=2048_batch=128\Mixed Gaussian_rho1=0.9


In [55]:
model_name="MINE_direct"
model_path = p.join(data_path, model_name)
print(model_path)

C:\Users\chung\Dropbox\GitHub\MINE-Entropy\minee\experiment\pop=2048_batch=128\Mixed Gaussian_rho1=0.9\MINE_direct


In [48]:
# os.makedirs(model_path,exist_ok=True)

# Import minee and needed packages

In [49]:
import minee 
import torch

In [51]:
from minee import settings
MINE_dir = settings.model['MINE_direct']['model']

# Load Mine_net

In [56]:
iter_num = 78125
mine_file = p.join(model_path, "checkpoint_iter={}.pt".format(iter_num))
MINE_dir.mine_net.load_state_dict(torch.load(mine_file))

# Specify data parameters

In [57]:
#Default value of datasets are shown
#Data can also be user-specified
def generate_dataset(selected_data):
    if (selected_data == 'Mixed Gaussian'):
        dataset = minee.data.mix_gaussian.MixedGaussian(n_samples=400, mean1=0, mean2=0, rho1=0.9, rho2=-0.9, mix=0.5, theta=0)
    elif (selected_data == 'Gaussian'):
        dataset = minee.data.gaussian.Gaussian(n_samples=200, mean1=0, mean2=1, rho=0.8)
    elif (selected_data == 'Mixed Uniform'):
        dataset = minee.data.mix_uniform.MixedUniform(mix=0.5, width_a=10, width_b=10, n_samples=200)
    elif (selected_data == 'Self-Defined'):
        pass
        #define custom dataset, n by 2 matrix with n being number of samples
        #can also define ground truth
        #dataset = 
    return dataset
gaus = generate_dataset('Gaussian')
X = gaus.data

# Load Train and valid mi_lb curve

In [58]:
import numpy as np
MINE_dir.avg_train_mi_lb = np.loadtxt(p.join(model_path, "avg_train_mi_lb.txt"))
MINE_dir.avg_valid_mi_lb = np.loadtxt(p.join(model_path, "avg_valid_mi_lb.txt"))
# MINE_dir.avg_train_loss = np.loadtxt(p.join(model_path, "avg_train_loss.txt"))
# MINE_dir.avg_test_loss = np.loadtxt(p.join(model_path, "avg_test_loss.txt"))

# Generate figure

In [59]:
from sklearn.model_selection import train_test_split
MINE_dir.X_train, MINE_dir.X_test = train_test_split(X, test_size=0.35, random_state=0)
MINE_dir.ground_truth = gaus.ground_truth
mi_lb = MINE_dir.forward_pass(MINE_dir.X_test).item()
MINE_dir.savefig(X, mi_lb, suffix="_test2")